In [1]:
%load_ext autoreload
%autoreload 2

import os


import sys

import scipy
import scipy.io
import time

import os.path
import warnings
warnings.filterwarnings('ignore')
os.environ["KMP_WARNINGS"] = "FALSE"


import tensorflow as tf
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

CHECKPOINT_PATH = "/home/vlq26735/code/PhD/GradientPathologiesPINNs/checkpoints/1DPoisson/"
MODE = "M1"

src_path = os.path.abspath(os.path.join('../../'))
print(f"src_path= {src_path}")

if src_path not in sys.path:
    sys.path.insert(0, src_path)


# op_dir, log_file = common.make_output_dir(CHECKPOINT_PATH, MODE)
# print(f"op_dir = {op_dir} \nlog_file = {log_file}")

from src.cases.Poisson import M1



src_path= /home/vlq26735/code/PhD/GradientPathologiesPINNs


In [2]:
class Sampler:
    # Initialize the class
    def __init__(self, dim, coords, func, name=None):
        self.dim = dim
        self.coords = coords
        self.func = func
        self.name = name

    def sample(self, N):
        x = self.coords[0:1, :] + (self.coords[1:2, :] - self.coords[0:1, :]) * np.random.rand(N, self.dim)
        y = self.func(x)
        return x, y

In [3]:
# Define solution and its Laplace
a = 4

def u(x, a):
  return np.sin(np.pi * a * x)

def u_xx(x, a):
  return -(np.pi * a)**2 * np.sin(np.pi * a * x)

In [4]:
# Define computional domain
bc1_coords = np.array([[0.0], [0.0]])
bc2_coords = np.array([[1.0], [1.0]])
dom_coords = np.array([[0.0], [1.0]])

# Training data on u(x) -- Dirichlet boundary conditions

nn  = 100

X_bc1 = dom_coords[0, 0] * np.ones((nn // 2, 1))
X_bc2 = dom_coords[1, 0] * np.ones((nn // 2, 1))
X_u = np.vstack([X_bc1, X_bc2])
Y_u = u(X_u, a)

X_r = np.linspace(dom_coords[0, 0], dom_coords[1, 0], nn)[:, None]
Y_r = u_xx(X_r, a)

nn = 1000
X_star = np.linspace(dom_coords[0, 0], dom_coords[1, 0], nn)[:, None]
u_star = u(X_star, a)
r_star = u_xx(X_star, a)

nIter =40001
bcbatch_size = 500
ubatch_size = 5000
mbbatch_size = 128



# Define model
mode = 'M1'
layers = [1, 500, 500 , 1]
starter_learning_rate = 1.0e-5

# Create residual sampler
gpu_options = tf.GPUOptions(visible_device_list="0")
tf.reset_default_graph()
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options,allow_soft_placement=False, log_device_placement=False))
# with  as sess:

model = M1.Possion_M1(layers, X_u, Y_u, X_r, Y_r , mode , starter_learning_rate , CHECKPOINT_PATH , sess)    


model.print("Using mode: " , model.mode)
model.print("neural network: " , model.layers )
model.print("Bc1 Training data size : " ,X_bc1.shape[0])
model.print("Bc2 Training data size : " ,X_bc2.shape[0])
model.print("interior Training data size : " ,X_r.shape[0])

model.print("Activation function: " , activFun)
model.print("number of iterations: " , nIter)
model.print("starter_learning_rate: " , starter_learning_rate)

model.print("Method desciption : gradual learing rate , with  " , method ," batch. ")


2024-02-27 15:52:43.801090: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-27 15:52:43.882755: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2445400000 Hz
2024-02-27 15:52:43.883473: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b20efdbae0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-02-27 15:52:43.883509: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-02-27 15:52:43.883709: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Using mode: M1
neural network: [1, 500, 500, 1]
Bc1 Training data size : 50
Bc2 Training data size : 50
interior Training data size : 100
Activation function: tanh
number of iterations: 40001
starter_learning_rate: 1.0000e-05
Method desciption : gradual learing rate , with  mini_batch batch. 


In [5]:

# Train model
start_time = time.time()

print("mini_batch method is used")
model.trainmb(nIter, mbbatch_size)
elapsed = time.time() - start_time

# Predictions
u_pred = model.predict_u(X_star)
r_pred = model.predict_r(X_star)
# Predictions

error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
error_r = np.linalg.norm(r_star - r_pred, 2) / np.linalg.norm(r_star, 2)

model.print('elapsed: {:.2e}'.format(elapsed))

model.print('Relative L2 error_u: {:.2e}'.format(error_u))
model.print('Relative L2 error_r: {:.2e}'.format(error_r))


model.print('mean value of lambda_bc: {:.2e}'.format(np.average(model.adaptive_constant_bcs_log)))
model.print('first value of lambda_bc: {:.2e}'.format(model.adaptive_constant_bcs_log[0]))
model.print('Relative L2 error_u: {:.2e}'.format(error_u))
model.print('Relative L2 error_v: {:.2e}'.format(error_r))

model.save_NN()
model.plot_grad()
model.plot_lambda()
model.plt_prediction( X_star , u_star , u_pred)
# sess.close()  
    

mini_batch method is used


It: 0, Loss: 1.234e+04, Loss_bcs: 3.020e-01, Loss_res: 1.234e+04 ,Time: 17.87
mean_grad_bcs: 1.9829e-01
mean_grad_res: 1.5759e-01
Gradients information stored ...
It: 100, Loss: 1.234e+04, Loss_bcs: 2.299e-02, Loss_res: 1.234e+04 ,Time: 75.75
Gradients information stored ...
It: 200, Loss: 1.234e+04, Loss_bcs: 1.047e-01, Loss_res: 1.234e+04 ,Time: 113.97
It: 300, Loss: 1.233e+04, Loss_bcs: 3.189e-01, Loss_res: 1.233e+04 ,Time: 125.99
It: 400, Loss: 1.231e+04, Loss_bcs: 8.440e-01, Loss_res: 1.231e+04 ,Time: 136.06
It: 500, Loss: 1.228e+04, Loss_bcs: 1.980e+00, Loss_res: 1.228e+04 ,Time: 142.64
It: 600, Loss: 1.223e+04, Loss_bcs: 4.071e+00, Loss_res: 1.222e+04 ,Time: 148.46
It: 700, Loss: 1.213e+04, Loss_bcs: 7.410e+00, Loss_res: 1.212e+04 ,Time: 153.63
It: 800, Loss: 1.198e+04, Loss_bcs: 1.201e+01, Loss_res: 1.197e+04 ,Time: 159.25
It: 900, Loss: 1.180e+04, Loss_bcs: 1.762e+01, Loss_res: 1.179e+04 ,Time: 167.22
It: 1000, Loss: 1.162e+04, Loss_bcs: 2.377e+01, Loss_res: 1.159e+04 ,Time: 1

KeyboardInterrupt: 

In [6]:
elapsed = time.time() - start_time

# Predictions
u_pred = model.predict_u(X_star)
r_pred = model.predict_r(X_star)
# Predictions

error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
error_r = np.linalg.norm(r_star - r_pred, 2) / np.linalg.norm(r_star, 2)

model.print('elapsed: {:.2e}'.format(elapsed))

model.print('Relative L2 error_u: {:.2e}'.format(error_u))
model.print('Relative L2 error_r: {:.2e}'.format(error_r))


model.print('mean value of lambda_bc: {:.2e}'.format(np.average(model.adaptive_constant_bcs_log)))
model.print('first value of lambda_bc: {:.2e}'.format(model.adaptive_constant_bcs_log[0]))
model.print('Relative L2 error_u: {:.2e}'.format(error_u))
model.print('Relative L2 error_v: {:.2e}'.format(error_r))

model.save_NN()
model.plot_grad()
model.plot_lambda()
model.plt_prediction( X_star , u_star , u_pred)

elapsed: 6.74e+02
Relative L2 error_u: 5.40e+00
Relative L2 error_r: 4.38e-02
mean value of lambda_bc: 1.00e+00
first value of lambda_bc: 1.00e+00
Relative L2 error_u: 5.40e+00
Relative L2 error_v: 4.38e-02
Save uv NN parameters successfully in %s .../home/vlq26735/code/PhD/GradientPathologiesPINNs/checkpoints/1DPoisson/Feb-27-2024_15-52-43-884584_M1
Final loss total loss: 7.398418e+01
Final loss loss_res: 3.064724e+01
Final loss loss_bcs: 4.333695e+01


In [17]:
import pickle
import pandas as pd
file = open("/home/vlq26735/code/PhD/GradientPathologiesPINNs/checkpoints/1DPoisson/Feb-26-2024_18-42-17-186118_M1/model.pickle", 'rb')

data  = pickle.load(file)

In [22]:
pdData = pd.DataFrame(data)
pdData[0][0].shape

(1, 500)

In [23]:
pdData[0][1].shape

(1, 500)

In [25]:
pdData[1][1].shape

(1, 500)

In [26]:
pdData[1][0].shape

(500, 500)